# Fábrica de Snapshots (Snapshot Factory)

Este notebook foi desenhado para rodar no **Google Colab** (com GPU).
Ele cria a estrutura de pastas recomendada e treina modelos Generalistas e Especialistas em lote.

> **Dica Colab**: Descomente a linha de instalação apropriada (Pública ou Privada) abaixo.

In [ ]:
# --- INSTALAÇÃO ---
# Descomente a linha abaixo para instalar no Colab/Kaggle
# !pip install --force-reinstall git+https://github.com/aretw0/predicoes-loterias-caixa.git


In [ ]:
import os
import sys
import pandas as pd

# --- LÓGICA DE IMPORTAÇÃO ---
from loterias.megasena import MegaSena
from loterias.snapshot_manager import SnapshotManager
print("✅ Módulos importados com sucesso. SnapshotManager Pronto.")

In [ ]:
# 2. Criar Estrutura de Pastas
games = ['megasena', 'lotofacil', 'quina']
types = ['generalistas', 'especialistas/virada', 'especialistas/acumulados']

for game in games:
    for t in types:
        path = f"snapshots/{game}/{t}"
        os.makedirs(path, exist_ok=True)
        print(f"Criado: {path}")

In [ ]:
# 3. Carregar Dados Reais
lottery = MegaSena()
df_full = lottery.preprocess_data()
print(f"Total Concursos: {len(df_full)}")

## 4. Treinar Generalistas (Todos os dados)
Use parâmetros agressivos aqui.

In [ ]:
# --- 4. Cultivo Generalista (Todos os Modelos) ---
# Treina Transformer, LSTM, CatBoost e AutoEncoder com TODO o histórico.
# Detecta GPU automaticamente.

manager = SnapshotManager(MegaSena())
manager.cultivate_generalists(epochs=200)

## 5. Treinar Especialistas (Mega da Virada / Final 0 ou 5)
Filtramos o DataFrame para conter apenas concursos com final 0 ou 5.

In [ ]:
# --- 5. Cultivo Especialista (Acumulados / Final 0 ou 5) ---
# Treina modelos focados apenas em concursos de final 0 ou 5.

def filtro_acumulados(df):
    if 'Concurso' in df.columns:
        return df[df['Concurso'] % 5 == 0].copy()
    return df[df.index % 5 == 0].copy()

manager.cultivate_specialists(
    filter_name="virada", 
    filter_func=filtro_acumulados, 
    epochs=200
)

In [ ]:
# 6. Baixar Snapshots (Google Colab)
# Compacta e baixa a pasta de snapshots automaticamente
import shutil
try:
    from google.colab import files
    shutil.make_archive('snapshots_backup', 'zip', 'snapshots')
    files.download('snapshots_backup.zip')
except ImportError:
    print("Ambiente não é o Colab ou google.colab não encontrado.")